reference : https://medium.com/nlplanet/two-minutes-nlp-beginner-intro-to-hugging-face-main-classes-and-functions-fb6a1d5579c4

In [ ]:
!pip install transformers datasets

## Pipline

In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

results = classifier("I'm so happy today!")
print(f"{results[0]['label']} with score {results[0]['score']}")
# POSITIVE with score 0.9998742341995239

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

POSITIVE with score 0.9998742341995239


## Dataset

In [4]:
import datasets

dataset = datasets.load_dataset("glue", "sst2")
print(dataset)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [7]:
dataset = datasets.load_dataset("glue", "sst2", split='train')
print(dataset)

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})


In [10]:
import pandas as pd

df = pd.DataFrame(dataset)
df.head()

,sentence,label,idx
0,hide new secretions from the parental units,0,0
1,"contains no wit , only labored gags",0,1
2,that loves its characters and communicates som...,1,2
3,remains utterly satisfied to remain the same t...,0,3
4,on the worst revenge-of-the-nerds clichés the ...,0,4


In [11]:
classifier = pipeline("sentiment-analysis")
# classifier = pipeline("sentiment-analysis", device=0) # predict using GPU
%time results = classifier(dataset.data["sentence"].to_pylist()[:500])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


CPU times: user 29.4 s, sys: 34.6 ms, total: 29.4 s
Wall time: 30.4 s


## Metrics

In [12]:
metric = datasets.load_metric("glue", "sst2")

n_samples = 500

X = dataset.data["sentence"].to_pylist()[:n_samples]
y = dataset.data["label"].to_pylist()[:n_samples]

results = classifier(X)
predictions = [0 if res["label"] == "NEGATIVE" else 1 for res in results]

print(metric.compute(predictions=predictions, references=y))

<ipython-input-12-4bf4521016a8>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("glue", "sst2")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'accuracy': 0.988}


## AutoClasses

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [14]:
encoding = tokenizer(["Hello!", "How are you?"], padding=True,
                     truncation=True, max_length=512, return_tensors="pt")
print(encoding)

{'input_ids': tensor([[  101, 29155,   106,   102,     0,     0],
        [  101, 12548, 10320, 10855,   136,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1]])}


In [15]:
outputs = model(**encoding)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2410, -0.9115, -0.3269, -0.0462,  1.2899],
        [-0.3575, -0.6521, -0.4409,  0.0471,  0.9552]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [16]:
from torch import nn

pt_predictions = nn.functional.softmax(outputs.logits, dim=-1)
print(pt_predictions)

tensor([[0.1210, 0.0619, 0.1110, 0.1470, 0.5592],
        [0.1269, 0.0945, 0.1168, 0.1902, 0.4716]], grad_fn=<SoftmaxBackward0>)


## Save and load models locally

In [ ]:
pt_save_directory = "./model"
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./model")